# classiq

I will start with small 2 qubit problems, one solvable and one not.

In [1]:
solvable_problem = [1,1] #An solvable instance of the partitioning problem to be solved using 2 quibts
unsolvable_problem = [1,3] #An unsolvable instance of the partitioning problem to be solved using 2 quibts

In [2]:
%pip install qiskit
%pip install classiq
%pip install nest_asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.7 MB 5.1 MB/s 
     |████████████████████████████████| 18.0 MB 43.4 MB/s 
     |████████████████████████████████| 240 kB 71.7 MB/s 
     |████████████████████████████████| 112 kB 74.0 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 37.5 MB 1.3 MB/s 
     |████████████████████████████████| 943 kB 69.1 MB/s 
     |████████████████████████████████| 49 kB 6.3 MB/s 
     |████████████████████████████████| 1.6 MB 40.0 MB/s 
     |████████████████████████████████| 49 kB 4.8 MB/s 
     |████████████████████████████████| 4.1 MB 46.2 MB/s 
     |████████████████████████████████| 112 kB 68.7 MB/s 
  Created wheel for qiskit: filename=qiskit-0.37.1-py3-none-any.whl size=12067 sha256=1b7579631c0c481a24db13e9b2b4cfeb5f181455e5f8f90733626628feb14543
  Stored in directory: /root/.cache/pip/wheels/1c/3c/a3/bf8c

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import nest_asyncio
nest_asyncio.apply()
import classiq

classiq.authenticate()

Your user code: TJJM-ZWVX
If a browser doesn't automatically open, please visit the url: https://classiq.eu.auth0.com/activate?user_code=TJJM-ZWVX


In [4]:
from classiq import ModelDesigner, Executor
from classiq.builtin_functions import UCC, HartreeFock

from classiq.interface.chemistry.molecule import Molecule
from classiq.interface.chemistry.ground_state_problem import GroundStateProblem
from classiq.interface.backend.backend_preferences import (
    IBMBackendPreferences,
    IBMBackendProvider,
)
from classiq.interface.executor.execution_preferences import ExecutionPreferences, OptimizerPreferences
from classiq.interface.executor.hamiltonian_minimization_problem import HamiltonianMinimizationProblem
from classiq.interface.executor.quantum_instruction_set import QuantumInstructionSet
from classiq.interface.executor.quantum_program import QuantumProgram
from classiq.interface.chemistry.operator import PauliOperator
from classiq import ModelDesigner
from classiq.builtin_functions import HardwareEfficientAnsatz

## preparing the hamiltonian for 2 quibts (Ising model)

In [5]:
def create_hamiltonian(S):
  n_1 = S[0]
  n_2 = S[1]

  H = [("II",n_1*n_1),
 ("ZZ",n_1*n_2),
 ("ZZ",n_2*n_1),
 ("II",n_2*n_2)]
  return H


## Prepare simulator

In [6]:
def define_simulator(simulator_name):
    if(simulator_name == "qasm" or simulator_name == "QASM"):
      curr_simulator = "qasm_simulator"
    else:
      curr_simulator = "aer_simulator_statevector"
    return curr_simulator

In [7]:
def define_optimizer(optimizer_name):
    if(optimizer_name == "cobyla" or optimizer_name == "COBYLA"):
      curr_optimizer = "COBYLA"
    else:
      curr_optimizer = "L_BFGS_B"
    return curr_optimizer

In [14]:
def prepare_simulator_classiq(curr_simulator, curr_optimizer,num_of_qubits=2, anzatz_first_gate = "x",anzatz_second_gate = "ry"):
    NUM_QUBITS = num_of_qubits
    FULLY_CONNECTED_MESH = []
    for i in range(num_of_qubits-1):
        FULLY_CONNECTED_MESH.append([i,i+1])
    hwea_params = HardwareEfficientAnsatz(
        num_qubits=NUM_QUBITS,
        connectivity_map=FULLY_CONNECTED_MESH,
        one_qubit_gates=[anzatz_first_gate, anzatz_second_gate],
        two_qubit_gates="cx",
        reps=3,
    )

    model_designer = ModelDesigner()
    model_designer.HardwareEfficientAnsatz(params=hwea_params)
    circuit = model_designer.synthesize()
    #circuit.show()
    #circuit.draw()
    circuit_qasm = circuit.qasm
    optimizer_preferences = OptimizerPreferences(
        max_iteration=60,
        name = curr_optimizer
    )

    ibm_provider = IBMBackendProvider(
        #insert details
    )

    backend_preferences = IBMBackendPreferences(
        backend_name=curr_simulator,
        access_token= "insert details",
        provider=ibm_provider,
    )

    num_shots = 8192

    execution_preferences = ExecutionPreferences(
        num_shots=num_shots,
        backend_preferences=backend_preferences,
        optimizer_preferences=optimizer_preferences
    )
    return execution_preferences, circuit_qasm

## compute minimal eigenvalue

In [9]:
def minimum_eigenvalue_by_vqe(H):
  hamiltonian_problem = HamiltonianMinimizationProblem(
      ansatz=QuantumProgram(code=circuit_qasm, syntax=QuantumInstructionSet.QASM),
      hamiltonian=H
  )
  result = Executor(
      preferences=execution_preferences
  ).execute_hamiltonian_minimization(hamiltonian_problem)
  print("minimal eigenvalue: ", format(result.energy,".3f"))

## compute minimal eigenvalue for solvable problem

In [10]:
partitioningProblem = create_hamiltonian(solvable_problem)
operator= PauliOperator(pauli_list=partitioningProblem)
print(operator)

pauli_list=[('II', (1+0j)), ('ZZ', (1+0j)), ('ZZ', (1+0j)), ('II', (1+0j))]


### qasm simulator and cobyla optimizer

stuck in a loop or returns error :/

In [11]:
# curr_simulator = define_simulator("qasm")
# curr_optimizer = define_optimizer("cobyla")
# execution_preferences, circuit_qasm = prepare_simulator_classiq(curr_simulator, curr_optimizer)
# minimum_eigenvalue_by_vqe(operator)

### qasm simulator and bfgs optimizer

stuck in a loop or returns error :/

In [12]:
# curr_simulator = define_simulator("qasm")
# curr_optimizer = define_optimizer("bfgs")
# execution_preferences, circuit_qasm = prepare_simulator_classiq(curr_simulator, curr_optimizer)
# minimum_eigenvalue_by_vqe(operator)

### aer_statevector simulator and cobyla optimizer - x anzays

In [15]:
curr_simulator = define_simulator("aer_statevector")
curr_optimizer = define_optimizer("cobyla")
execution_preferences, circuit_qasm = prepare_simulator_classiq(curr_simulator, curr_optimizer)
minimum_eigenvalue_by_vqe(operator)

minimal eigenvalue:  0.000


### aer_statevector simulator and cobyla optimizer - RyRz anzats

In [16]:
curr_simulator = define_simulator("aer_statevector")
curr_optimizer = define_optimizer("cobyla")
execution_preferences, circuit_qasm = prepare_simulator_classiq(curr_simulator, curr_optimizer,2 , "ry","rz")
minimum_eigenvalue_by_vqe(operator)

minimal eigenvalue:  0.000


### aer_statevector simulator and bfgs optimizer - x anzats

returns wrong result :/ expected result: 0.000

In [17]:
curr_simulator = define_simulator("aer_statevector")
curr_optimizer = define_optimizer("bfgs")
execution_preferences, circuit_qasm = prepare_simulator_classiq(curr_simulator, curr_optimizer)
minimum_eigenvalue_by_vqe(operator)

minimal eigenvalue:  0.001


### aer_statevector simulator and bfgs optimizer - RyRz anzats

returns wrong result :/ expected result: 0.000

In [18]:
curr_simulator = define_simulator("aer_statevector")
curr_optimizer = define_optimizer("bfgs")
execution_preferences, circuit_qasm = prepare_simulator_classiq(curr_simulator, curr_optimizer,2,"ry","rz")
minimum_eigenvalue_by_vqe(operator)

minimal eigenvalue:  0.001


## compute minimal eigenvalue for unsolvable problem

In [19]:
partitioningProblem = create_hamiltonian(unsolvable_problem)
operator= PauliOperator(pauli_list=partitioningProblem)
print(operator)

pauli_list=[('II', (1+0j)), ('ZZ', (3+0j)), ('ZZ', (3+0j)), ('II', (9+0j))]


### qasm simulator and cobyla optimizer

stuck in a loop or returns error :/

In [ ]:
# curr_simulator = define_simulator("qasm")
# curr_optimizer = define_optimizer("cobyla")
# execution_preferences, circuit_qasm = prepare_simulator_classiq(curr_simulator, curr_optimizer)
# minimum_eigenvalue_by_vqe(operator)

### qasm simulator and bfgs optimizer

stuck in a loop or returns error :/

In [ ]:
# curr_simulator = define_simulator("qasm")
# curr_optimizer = define_optimizer("bfgs")
# execution_preferences, circuit_qasm = prepare_simulator_classiq(curr_simulator, curr_optimizer)
# minimum_eigenvalue_by_vqe(operator)

### aer_statevector simulator and cobyla optimizer - x anzats

In [20]:
curr_simulator = define_simulator("aer_statevector")
curr_optimizer = define_optimizer("cobyla")
execution_preferences, circuit_qasm = prepare_simulator_classiq(curr_simulator, curr_optimizer)
minimum_eigenvalue_by_vqe(operator)

minimal eigenvalue:  4.000


### aer_statevector simulator and cobyla optimizer - RyRz anzats

returns wrong result :/ expected result: 4.000

In [21]:
curr_simulator = define_simulator("aer_statevector")
curr_optimizer = define_optimizer("cobyla")
execution_preferences, circuit_qasm = prepare_simulator_classiq(curr_simulator, curr_optimizer, 2, "ry","rz")
minimum_eigenvalue_by_vqe(operator)

minimal eigenvalue:  4.001


### aer_statevector simulator and bfgs optimizer - x anzats

returns wrong result :/ expected result: 4.000

In [22]:
curr_simulator = define_simulator("aer_statevector")
curr_optimizer = define_optimizer("bfgs")
execution_preferences, circuit_qasm = prepare_simulator_classiq(curr_simulator, curr_optimizer)
minimum_eigenvalue_by_vqe(operator)

minimal eigenvalue:  4.003


### aer_statevector simulator and bfgs optimizer - RyRz anzats

returns wrong result :/ expected result: 4.000

In [24]:
curr_simulator = define_simulator("aer_statevector")
curr_optimizer = define_optimizer("bfgs")
execution_preferences, circuit_qasm = prepare_simulator_classiq(curr_simulator, curr_optimizer,2,"ry","rz")
minimum_eigenvalue_by_vqe(operator)

minimal eigenvalue:  4.003


## General Form

In [25]:
solvable_problem_4_qubits = [1,2,3,2]
solvable_problem_3_qubits = [1,2,1]
unsolvable_problem_3_qubits = [1,3,1]
unsolvable_problem_4_qubits = [1,2,4,2]

### preparing the hamiltonian for n quibts (Ising model)

In [26]:
import numpy as np
from qiskit.opflow import Z, I

def multiply_ising_models(ham1, ham2, coef1, coef2):
  final_sigma = ""
  final_coef = 0
  for i in range(len(ham1)):
      sigma1 = ham1[i]
      sigma2 = ham2[i]
      if (sigma1 == 'Z' and sigma2 == 'Z'):
        final_sigma += 'I'
      elif (sigma1 == 'I' and sigma2 == 'Z'):
        final_sigma += 'Z'
      elif (sigma1 == 'Z' and sigma2 == 'I'):
        final_sigma += 'Z'
      else:
        final_sigma += 'I'
  return (final_sigma,coef1*coef2)


def create_general_hamiltonian_classiq(S):
  num_qubits = len(S)
  operator = []
  sigmas_list = []
  for qubit in range(num_qubits):
    if qubit == 0:
       sigma = "Z"
    else:
       sigma = "I"
    for i in range(1,num_qubits):
      if i == qubit:
        sigma = sigma + "Z"
      else:
        sigma = sigma + "I"
    sigmas_list.append((sigma,S[qubit]))

  for i in range(len(sigmas_list)):
    for j in range(len(sigmas_list)):
      sigma = multiply_ising_models(sigmas_list[i][0], sigmas_list[j][0], sigmas_list[i][1], sigmas_list[j][1])
      operator.append(sigma)
  return operator 


### Example: solvable 3 qubit problem

In [27]:
H = create_general_hamiltonian_classiq(solvable_problem_3_qubits)
operator= PauliOperator(pauli_list=H)
print(operator)

pauli_list=[('III', (1+0j)), ('ZZI', (2+0j)), ('ZIZ', (1+0j)), ('ZZI', (2+0j)), ('III', (4+0j)), ('IZZ', (2+0j)), ('ZIZ', (1+0j)), ('IZZ', (2+0j)), ('III', (1+0j))]


### aer_statevector simulator and cobyla optimizer - x anzats

returns wrong result :/ expected result: 0.000

In [28]:
curr_simulator = define_simulator("aer_statevector")
curr_optimizer = define_optimizer("cobyla")
execution_preferences, circuit_qasm = prepare_simulator_classiq(curr_simulator, curr_optimizer,3)
minimum_eigenvalue_by_vqe(operator)

minimal eigenvalue:  0.006


### aer_statevector simulator and cobyla optimizer - RyRz anzats

returns wrong result :/ expected result: 0.000

In [29]:
curr_simulator = define_simulator("aer_statevector")
curr_optimizer = define_optimizer("cobyla")
execution_preferences, circuit_qasm = prepare_simulator_classiq(curr_simulator, curr_optimizer,3, "ry","rz")
minimum_eigenvalue_by_vqe(operator)

minimal eigenvalue:  0.056


### aer_statevector simulator and bfgs optimizer - x anzats

returns wrong result :/ expected result: 0.000

In [ ]:
curr_simulator = define_simulator("aer_statevector")
curr_optimizer = define_optimizer("bfgs")
execution_preferences, circuit_qasm = prepare_simulator_classiq(curr_simulator, curr_optimizer,3)
minimum_eigenvalue_by_vqe(operator)

minimal eigenvalue:  0.011


### aer_statevector simulator and bfgs optimizer - RyRz anzats

returns wrong result :/ expected result: 0.000

In [30]:
curr_simulator = define_simulator("aer_statevector")
curr_optimizer = define_optimizer("bfgs")
execution_preferences, circuit_qasm = prepare_simulator_classiq(curr_simulator, curr_optimizer,3,"ry","rz")
minimum_eigenvalue_by_vqe(operator)

minimal eigenvalue:  0.022


### Example: unsolvable 3 qubit problem

In [32]:
H = create_general_hamiltonian_classiq(unsolvable_problem_3_qubits)
operator= PauliOperator(pauli_list=H)
print(operator)

pauli_list=[('III', (1+0j)), ('ZZI', (3+0j)), ('ZIZ', (1+0j)), ('ZZI', (3+0j)), ('III', (9+0j)), ('IZZ', (3+0j)), ('ZIZ', (1+0j)), ('IZZ', (3+0j)), ('III', (1+0j))]


### aer_statevector simulator and cobyla optimizer - x anzats

returns wrong result :/ expected result: 1.000

In [33]:
curr_simulator = define_simulator("aer_statevector")
curr_optimizer = define_optimizer("cobyla")
execution_preferences, circuit_qasm = prepare_simulator_classiq(curr_simulator, curr_optimizer,3)
minimum_eigenvalue_by_vqe(operator)

minimal eigenvalue:  1.004


### aer_statevector simulator and cobyla optimizer - RyRz anzats

returns wrong result :/ expected result: 1.000

In [34]:
curr_simulator = define_simulator("aer_statevector")
curr_optimizer = define_optimizer("cobyla")
execution_preferences, circuit_qasm = prepare_simulator_classiq(curr_simulator, curr_optimizer,3,"ry","rz")
minimum_eigenvalue_by_vqe(operator)

minimal eigenvalue:  1.174


### aer_statevector simulator and bfgs optimizer - x anzats

returns wrong result :/ expected result: 1.000

In [35]:
curr_simulator = define_simulator("aer_statevector")
curr_optimizer = define_optimizer("bfgs")
execution_preferences, circuit_qasm = prepare_simulator_classiq(curr_simulator, curr_optimizer,3)
minimum_eigenvalue_by_vqe(operator)

minimal eigenvalue:  1.005


### aer_statevector simulator and bfgs optimizer - RyRz anzats

returns wrong result :/ expected result: 1.000

In [36]:
curr_simulator = define_simulator("aer_statevector")
curr_optimizer = define_optimizer("bfgs")
execution_preferences, circuit_qasm = prepare_simulator_classiq(curr_simulator, curr_optimizer,3, "ry","rz")
minimum_eigenvalue_by_vqe(operator)

minimal eigenvalue:  1.032
